# 05 - AutoML 工具对比实验

> 🎯 学习目标：横向对比PyCaret、Auto-sklearn、FLAML、H2O，理解各工具的优劣

---

## 📚 本节内容

1. 对比维度设计
2. 统一数据集准备
3. 各工具性能测试
4. 速度对比
5. 准确率对比
6. 易用性对比
7. 综合评分
8. 选型建议

---

## 🎯 对比维度

| 维度 | 说明 |
|------|------|
| **准确率** | 在测试集上的性能 |
| **速度** | 训练时间 |
| **易用性** | 代码复杂度 |
| **功能丰富度** | 支持的功能数量 |
| **可视化** | 图表和分析工具 |
| **可解释性** | 模型解释工具 |
| **企业级特性** | 分布式、部署等 |

---

## 1. 环境准备

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# 设置绘图风格
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

print("✅ 基础环境准备完成")

---

## 2. 统一数据集准备

In [ ]:
# 加载数据集
X, y = load_breast_cancer(return_X_y=True)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"数据集: 乳腺癌分类")
print(f"训练集: {X_train.shape}")
print(f"测试集: {X_test.shape}")
print(f"类别分布: {np.bincount(y_train)}")

---

## 3. PyCaret 测试

In [ ]:
print("=" * 50)
print("测试 1: PyCaret")
print("=" * 50)

from pycaret.classification import *

# 准备数据
data_pycaret = pd.DataFrame(X_train)
data_pycaret['target'] = y_train

# 计时开始
start_time = time.time()

# 初始化
clf = setup(
    data=data_pycaret,
    target='target',
    session_id=42,
    verbose=False
)

# 对比模型
best_pycaret = compare_models(n_select=1, verbose=False)

# 调优
tuned_pycaret = tune_model(best_pycaret, verbose=False)

# 计时结束
pycaret_time = time.time() - start_time

# 预测
y_pred_pycaret = predict_model(tuned_pycaret, data=pd.DataFrame(X_test))['prediction_label']

# 评估
pycaret_accuracy = accuracy_score(y_test, y_pred_pycaret)
pycaret_f1 = f1_score(y_test, y_pred_pycaret)

print(f"\n✅ PyCaret 完成")
print(f"  耗时: {pycaret_time:.2f}秒")
print(f"  准确率: {pycaret_accuracy:.4f}")
print(f"  F1 Score: {pycaret_f1:.4f}")
print(f"  最佳模型: {best_pycaret}")

---

## 4. Auto-sklearn 测试

In [ ]:
print("\n" + "=" * 50)
print("测试 2: Auto-sklearn")
print("=" * 50)

import autosklearn.classification

# 计时开始
start_time = time.time()

# 训练
automl_sklearn = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    seed=42
)
automl_sklearn.fit(X_train, y_train)

# 计时结束
sklearn_time = time.time() - start_time

# 预测
y_pred_sklearn = automl_sklearn.predict(X_test)

# 评估
sklearn_accuracy = accuracy_score(y_test, y_pred_sklearn)
sklearn_f1 = f1_score(y_test, y_pred_sklearn)

print(f"\n✅ Auto-sklearn 完成")
print(f"  耗时: {sklearn_time:.2f}秒")
print(f"  准确率: {sklearn_accuracy:.4f}")
print(f"  F1 Score: {sklearn_f1:.4f}")

---

## 5. FLAML 测试

In [ ]:
print("\n" + "=" * 50)
print("测试 3: FLAML")
print("=" * 50)

from flaml import AutoML

# 计时开始
start_time = time.time()

# 训练
automl_flaml = AutoML()
automl_flaml.fit(
    X_train, y_train,
    task='classification',
    time_budget=120,
    metric='accuracy',
    verbose=0,
    seed=42
)

# 计时结束
flaml_time = time.time() - start_time

# 预测
y_pred_flaml = automl_flaml.predict(X_test)

# 评估
flaml_accuracy = accuracy_score(y_test, y_pred_flaml)
flaml_f1 = f1_score(y_test, y_pred_flaml)

print(f"\n✅ FLAML 完成")
print(f"  耗时: {flaml_time:.2f}秒")
print(f"  准确率: {flaml_accuracy:.4f}")
print(f"  F1 Score: {flaml_f1:.4f}")
print(f"  最佳模型: {automl_flaml.best_estimator}")

---

## 6. H2O AutoML 测试

In [ ]:
print("\n" + "=" * 50)
print("测试 4: H2O AutoML")
print("=" * 50)

import h2o
from h2o.automl import H2OAutoML

# 初始化H2O
h2o.init(verbose=False)

# 准备数据
train_h2o = h2o.H2OFrame(pd.DataFrame(X_train))
train_h2o['target'] = h2o.H2OFrame(y_train.reshape(-1, 1))
train_h2o['target'] = train_h2o['target'].asfactor()

test_h2o = h2o.H2OFrame(pd.DataFrame(X_test))

# 计时开始
start_time = time.time()

# 训练
aml_h2o = H2OAutoML(
    max_runtime_secs=120,
    seed=42,
    verbosity='warn'
)
aml_h2o.train(
    x=list(range(X_train.shape[1])),
    y='target',
    training_frame=train_h2o
)

# 计时结束
h2o_time = time.time() - start_time

# 预测
predictions_h2o = aml_h2o.leader.predict(test_h2o)
y_pred_h2o = predictions_h2o['predict'].as_data_frame().values.ravel()

# 评估
h2o_accuracy = accuracy_score(y_test, y_pred_h2o)
h2o_f1 = f1_score(y_test, y_pred_h2o)

print(f"\n✅ H2O AutoML 完成")
print(f"  耗时: {h2o_time:.2f}秒")
print(f"  准确率: {h2o_accuracy:.4f}")
print(f"  F1 Score: {h2o_f1:.4f}")

# 关闭H2O
h2o.cluster().shutdown(prompt=False)

---

## 7. 对比结果汇总

In [ ]:
# 汇总结果
results = pd.DataFrame({
    'Tool': ['PyCaret', 'Auto-sklearn', 'FLAML', 'H2O'],
    'Time (s)': [pycaret_time, sklearn_time, flaml_time, h2o_time],
    'Accuracy': [pycaret_accuracy, sklearn_accuracy, flaml_accuracy, h2o_accuracy],
    'F1 Score': [pycaret_f1, sklearn_f1, flaml_f1, h2o_f1]
})

print("\n" + "=" * 60)
print("📊 AutoML 工具对比结果")
print("=" * 60)
print(results.to_string(index=False))
print("=" * 60)

---

## 8. 可视化对比

### 8.1 速度对比

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 速度对比
axes[0].barh(results['Tool'], results['Time (s)'])
axes[0].set_xlabel('Time (seconds)')
axes[0].set_title('Training Time Comparison')
axes[0].invert_yaxis()

# 准确率对比
axes[1].barh(results['Tool'], results['Accuracy'], color='orange')
axes[1].set_xlabel('Accuracy')
axes[1].set_title('Accuracy Comparison')
axes[1].set_xlim([0.9, 1.0])
axes[1].invert_yaxis()

# F1 Score对比
axes[2].barh(results['Tool'], results['F1 Score'], color='green')
axes[2].set_xlabel('F1 Score')
axes[2].set_title('F1 Score Comparison')
axes[2].set_xlim([0.9, 1.0])
axes[2].invert_yaxis()

plt.tight_layout()
plt.show()

### 8.2 综合评分雷达图

In [ ]:
# 综合评分（主观评分，根据实际使用体验）
categories = ['速度', '准确率', '易用性', '功能丰富度', '可视化', '可解释性', '企业级特性']

scores = {
    'PyCaret': [4, 5, 5, 5, 5, 4, 3],
    'Auto-sklearn': [3, 5, 3, 4, 2, 3, 3],
    'FLAML': [5, 4, 4, 3, 2, 3, 4],
    'H2O': [4, 5, 3, 5, 4, 5, 5]
}

# 绘制雷达图
angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

for tool, score in scores.items():
    values = score + score[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=tool)
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, size=12)
ax.set_ylim(0, 5)
ax.set_yticks([1, 2, 3, 4, 5])
ax.set_yticklabels(['1', '2', '3', '4', '5'])
ax.grid(True)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
plt.title('AutoML 工具综合评分雷达图', size=16, pad=20)
plt.show()

---

## 9. 选型建议

### 根据场景选择

```python
场景                        推荐工具            原因
────────────────────────────────────────────────────
快速原型开发                PyCaret            最易用，功能丰富
时间预算有限                FLAML              速度最快
需要深入理解模型            Auto-sklearn       贝叶斯优化，元学习
企业级生产部署              H2O AutoML         分布式，MOJO部署
需要丰富可视化              PyCaret            20+种图表
需要模型可解释性            H2O AutoML         SHAP，部分依赖图
sklearn用户                 Auto-sklearn       无缝集成
大数据集                    FLAML/H2O          采样策略，分布式
```

### 综合推荐

1. **入门首选**: **PyCaret** ⭐⭐⭐⭐⭐
   - 最易上手
   - 功能最全
   - 可视化最好

2. **速度优先**: **FLAML** ⭐⭐⭐⭐⭐
   - 速度最快
   - 成本最低
   - 适合快速迭代

3. **企业级**: **H2O AutoML** ⭐⭐⭐⭐
   - 分布式训练
   - 生产部署
   - 可解释性强

4. **学习理解**: **Auto-sklearn** ⭐⭐⭐⭐
   - 贝叶斯优化
   - 元学习
   - sklearn兼容

---

## 10. 实践建议

### 组合使用策略

```python
# 步骤1: 使用FLAML快速验证（1分钟）
from flaml import AutoML
automl = AutoML()
automl.fit(X_train, y_train, task='classification', time_budget=60)

# 步骤2: 如果效果好，使用PyCaret深入分析（10分钟）
from pycaret.classification import *
setup(data, target='target')
best = compare_models()
plot_model(best, plot='feature')  # 分析特征重要性

# 步骤3: 如果需要部署，使用H2O AutoML（30分钟）
import h2o
from h2o.automl import H2OAutoML
aml = H2OAutoML(max_runtime_secs=1800)
aml.train(x=features, y=target, training_frame=train)
aml.leader.download_mojo()  # 导出MOJO用于生产
```

### 学习路径

1. **第1天**: PyCaret快速上手
2. **第2天**: FLAML速度体验
3. **第3天**: Auto-sklearn理论理解
4. **第4天**: H2O企业级特性
5. **第5-7天**: 综合项目实战

---

## 11. 总结

### 核心结论

1. **没有"最好"的工具，只有"最合适"的工具**
2. **组合使用效果更好**：FLAML验证 → PyCaret分析 → H2O部署
3. **理解原理比使用工具更重要**
4. **AutoML是工具，不是银弹**

### 最终建议

- ✅ 学习阶段：**PyCaret**（易用性高，学习曲线平缓）
- ✅ 竞赛/快速迭代：**FLAML**（速度快，成本低）
- ✅ 企业项目：**H2O AutoML**（功能全，可部署）
- ✅ 深入理解：**Auto-sklearn**（算法先进，理论完备）

---

## 🎯 综合练习

1. **任务1**: 在自己的数据集上对比4个工具
2. **任务2**: 分析为什么某个工具在特定数据集上表现更好
3. **任务3**: 设计一个组合使用AutoML工具的workflow
4. **任务4**: 完成 `projects/01_baseline_comparison/` 项目

---

## 📚 参考资源

- [AutoML Benchmark](https://github.com/openml/automlbenchmark)
- [Papers with Code - AutoML](https://paperswithcode.com/task/automl)
- [AutoML Survey Paper](https://arxiv.org/abs/1908.00709)

---

**恭喜完成 AutoML 工具链学习！🎉**

**下一步**: 返回主学习路线，进入神经网络阶段，或在实际项目中应用AutoML